# 

# Notebook Summary 


### Quickstart

  1. Import etiq library - for install please check our docs (https://docs.etiq.ai/) 

  2. Login to the dashboard - this way you can send the results to your dashboard instance (Etiq AWS instance if you use the SaaS version). To deploy on your own cloud instance, get in touch (info@etiq.ai)

  3. Create or open a project 
  
### Example dataset & model


  4. Example model: using Adult dataset, predicts income above/below 50K 
  
### Create a snapshot 

  5. Log dataset & model 
  
  6. Load a config file
  
  6. Create a snapshot 
  
  
### Scan your snapshot 
  
  7. Run a scan 
  
  8. Retrieve results
  
  
### Example other scans 

# 

# Quickstart 

In [1]:
import etiq


Thanks for trying out the ETIQ.ai toolkit!

Visit our getting started documentation at https://docs.etiq.ai/

Visit our Slack channel at https://etiqcore.slack.com/ for support or feedback.



In [2]:
from etiq import login as etiq_login
etiq_login("https://dashboard.etiq.ai/", "<token>")


(Dashboard supplied updated license information)


Connection successful. Projects and pipelines will be displayed in the dashboard. 😀

In [3]:
# Can enumerate all available projects
all_projects = etiq.projects.get_all_projects()
print(all_projects)

[<ETIQ:Project [1] "Demo Project">, <ETIQ:Project [2] "Test Already Trained">]


In [4]:
# Can get/create a single named project
project = etiq.projects.open(name="Demo Already Trained")

# 

# Example dataset and model

To illustrate some of the library's features, we build a model that predicts whether an applicant makes over or under 50K using the Adult dataset from https://archive.ics.uci.edu/ml/datasets/adult.

1. Import the dataset

2. Pre-process the dataset 

3. Build the model 

4. Log the dataset and already built model to Etiq


In this case we encode prior to splitting into test/train/validate because we know in advance the categories people fall into for this dataset. This means that in production we won't run into new categories that will fall into a bucket not included in this dataset, This allows us to encode prior to splitting into train/test/validation. 

However if this is not the case for your use case, you should NOT encode prior to splitting your sample, as this might lead to LEAKAGE. 

Encoding categorical values itself is problematic as it assigns a numerical ranking to categorical variables. For best practice encoding use one hot encoding. As we limit the free library functionality to 15 features, we will not do one-hot encoding for the purposes of this example. 

Remember: This is an example only. The use case for the majority of scans in Etiq is that you log the model to Etiq once you have the sample that you'll be training on. Usually this sample will have numeric features only as otherwise you will not be able to use it in with the majority of supported libraries training methods. 

In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Loading a dataset. We're using the adult dataset
data = etiq.utils.load_sample("adultdata")
data.head()

# use a LabelEncoder to transform categorical variables
cont_vars = ['age', 'educational-num', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']
cat_vars = list(set(data.columns.values) - set(cont_vars))

label_encoders = {}
data_encoded = pd.DataFrame()
for i in cat_vars:
    label = LabelEncoder()
    data_encoded[i] = label.fit_transform(data[i])
    label_encoders[i] = label

data_encoded.set_index(data.index, inplace=True)
data_encoded = pd.concat([data.loc[:, cont_vars], data_encoded], axis=1).copy()



In [6]:
# prepare the training/testing/validation datasets

# separate into train/validate/test dataset of sizes 80%/10%/10% as percetages of the initial data
data_remaining, test = train_test_split(data_encoded, test_size=0.1)
train, valid = train_test_split(data_remaining, test_size=0.1112)

# because we don't want to train on protected attributes or labels to be predicted, 
# let's remove these columns from the training dataset
protected_train = train['gender'].copy() # gender is a protected attribute
y_train = train['income'].copy() # labels we're going to train the model to predict
x_train = train.drop(columns=['gender','income'])
protected_valid = valid['gender'].copy() 
y_valid = valid['income'].copy() 
x_valid = valid.drop(columns=['gender','income'])
protected_test = test['gender'].copy() 
y_test = test['income'].copy()
x_test = test.drop(columns=['gender','income'])

In [7]:
# train a XGBoost model to predict 'income'

standard_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=4)    
model_fit = standard_model.fit(x_train, y_train)

In [8]:
y_train_pred = standard_model.predict(x_train)
y_valid_pred = standard_model.predict(x_valid)
print('Model accuracy on the training dataset :', 
      round(100 * accuracy_score(y_train, y_train_pred),2),'%') # round the score to 2 digits  

print('Model accuracy on the validation dataset :', 
      round(100 * accuracy_score(y_valid, y_valid_pred),2),'%')

Model accuracy on the training dataset : 90.09 %
Model accuracy on the validation dataset : 86.75 %


# 

# Log dataset & model snapshot to Etiq 

This is an example of how you log the already trained model to Etiq.

When you log the dataset make sure you log a new sample or a test sample, not the sample you trained you model on. 

When you log the config, make sure the % for train and valid splits are 0. 

If you are planning to use this in production, not just pre-production get in touch with us. We will release integration demos shortly.


## Loading the config file 

The config is where you can set-up the scans you want, and the thresholds outside of which Etiq finds a problem. In the config you input relevant parameters (e.g. for bias/fairness scans you'll have to tell Etiq which feature is a demographic and which value represents a protected group). 
For more details on the config just check the documentation. You can upload these config files from wherever you want. We provide examples in the Demo repo with each notebook.   

The config gets stored in the database so you can have a log and version control. 

When you log the config for an already trained model, please log 



In [9]:

etiq.load_config("./config_already_trained.json")


{'dataset': {'label': 'income',
  'bias_params': {'protected': 'gender',
   'privileged': 1,
   'unprivileged': 0,
   'positive_outcome_label': 1,
   'negative_outcome_label': 0},
  'train_valid_test_splits': [0.0, 1.0, 0.0],
  'cat_col': 'cat_vars',
  'cont_col': 'cont_vars'},
 'scan_accuracy_metrics': {'thresholds': {'accuracy': [0.8, 1.0],
   'true_pos_rate': [0.6, 1.0],
   'true_neg_rate': [0.6, 1.0]}},
 'scan_bias_metrics': {'thresholds': {'equal_opportunity': [0.0, 0.2],
   'demographic_parity': [0.0, 0.2],
   'equal_odds_tnr': [0.0, 0.2],
   'individual_fairness': [0.0, 0.8],
   'equal_odds_tpr': [0.0, 0.2]}},
 'scan_leakage': {'leakage_threshold': 0.85}}

## Log the dataset and model

In [10]:
from etiq import Model


#log your dataset

dataset_loader = etiq.dataset(test)

#Log your already trained model

model = Model(model_architecture=standard_model, model_fitted=model_fit)



## Creating a snapshot

In [11]:
# Creating a snapshot
snapshot = project.snapshots.create(name="Test Snapshot", dataset=dataset_loader.initial_dataset, model=model, bias_params=dataset_loader.bias_params)



# 

# Run scans

You can run multiple scans for snapshot. 

## Bias Metrics 

In [12]:
#scan_bias_metrics

(segments, issues, issue_summary) = snapshot.scan_bias_metrics()

INFO:etiq.pipeline.BiasMetricsIssuePipeline0638:Starting pipeline
INFO:etiq.pipeline.BiasMetricsIssuePipeline0638:Computed bias metrics for the dataset
INFO:etiq.pipeline.BiasMetricsIssuePipeline0638:Completed pipeline


In [13]:
issues

""


In [14]:
issue_summary

,name,metric,measure,features,segments,total_issues_tested,issues_found,threshold
0,demographic_parity_below_threshold,<function demographic_parity at 0x7f8cef48d310>,None,{},{},1,0,"[0.0, 0.2]"
1,demographic_parity_above_threshold,<function demographic_parity at 0x7f8cef48d310>,None,{},{},1,0,"[0.0, 0.2]"
2,equal_odds_tpr_below_threshold,<function equal_odds_tpr at 0x7f8cef48d3a0>,None,{},{},1,0,"[0.0, 0.2]"
3,equal_odds_tpr_above_threshold,<function equal_odds_tpr at 0x7f8cef48d3a0>,None,{},{},1,0,"[0.0, 0.2]"
4,equal_odds_tnr_below_threshold,<function equal_odds_tnr at 0x7f8cef48d430>,None,{},{},1,0,"[0.0, 0.2]"
5,equal_odds_tnr_above_threshold,<function equal_odds_tnr at 0x7f8cef48d430>,None,{},{},1,0,"[0.0, 0.2]"
6,equal_opportunity_below_threshold,<function equal_opportunity at 0x7f8cef48d4c0>,None,{},{},1,0,"[0.0, 0.2]"
7,equal_opportunity_above_threshold,<function equal_opportunity at 0x7f8cef48d4c0>,None,{},{},1,0,"[0.0, 0.2]"
8,individual_fairness_below_threshold,<function individual_fairness at 0x7f8cef48d700>,None,{},{},1,0,"[0.0, 0.8]"
9,individual_fairness_above_threshold,<function individual_fairness at 0x7f8cef48d700>,None,{},{},1,0,"[0.0, 0.8]"


## Accuracy metrics scan

In [15]:
(segments, issues, issue_summary) = snapshot.scan_accuracy_metrics()

INFO:etiq.pipeline.AccuracyMetricsIssuePipeline0731:Starting pipeline
INFO:etiq.pipeline.AccuracyMetricsIssuePipeline0731:Computed acurracy metrics for the dataset
INFO:etiq.pipeline.AccuracyMetricsIssuePipeline0731:Completed pipeline


In [16]:
issue_summary

,name,metric,measure,features,segments,total_issues_tested,issues_found,threshold
0,accuracy_below_threshold,<function accuracy at 0x7f8cef48d160>,None,{},{},1,0,"[0.8, 1.0]"
1,accuracy_above_threshold,<function accuracy at 0x7f8cef48d160>,None,{},{},1,0,"[0.8, 1.0]"
2,true_pos_rate_below_threshold,<function true_pos_rate at 0x7f8cef48d1f0>,None,{},{},1,0,"[0.6, 1.0]"
3,true_pos_rate_above_threshold,<function true_pos_rate at 0x7f8cef48d1f0>,None,{},{},1,0,"[0.6, 1.0]"
4,true_neg_rate_below_threshold,<function true_neg_rate at 0x7f8cef48d280>,None,{},{},1,0,"[0.6, 1.0]"
5,true_neg_rate_above_threshold,<function true_neg_rate at 0x7f8cef48d280>,None,{},{},1,0,"[0.6, 1.0]"


# Bias Sources

Even if your training and test samples are very similar, the bias sources are related to properties of the training dataset, not the test dataset. 

To run bias sources even if you have an already trained model please use the TRAINING dataset instead.

You will need to change the train_test_valid in your config & then re-log your snapshots. You will still have the same model architecture from your already trained model, but if the scans show issues we recommend you clean your data and retrain the model. 

To be able to use bias sources you will have to log which features are categorical and which are numeric. 

You can do this in the config or in the notebook if easier. Commented out example shows you how to you do it in your notebook.



In [17]:
etiq.load_config("./config_already_trained_bias_sources.json")


{'dataset': {'label': 'income',
  'bias_params': {'protected': 'gender',
   'privileged': 1,
   'unprivileged': 0,
   'positive_outcome_label': 1,
   'negative_outcome_label': 0},
  'train_valid_test_splits': [1.0, 0.01, 0.0],
  'cat_col': ['workclass',
   'relationship',
   'occupation',
   'gender',
   'race',
   'native-country',
   'marital-status',
   'income',
   'education'],
  'cont_col': ['age',
   'educational-num',
   'fnlwgt',
   'capital-gain',
   'capital-loss',
   'hours-per-week']},
 'scan_bias_sources': {'auto': True},
 'scan_accuracy_metrics': {'thresholds': {'accuracy': [0.8, 1.0],
   'true_pos_rate': [0.6, 1.0],
   'true_neg_rate': [0.6, 1.0]}},
 'scan_bias_metrics': {'thresholds': {'equal_opportunity': [0.0, 0.2],
   'demographic_parity': [0.0, 0.2],
   'equal_odds_tnr': [0.0, 0.2],
   'individual_fairness': [0.0, 0.8],
   'equal_odds_tpr': [0.0, 0.2]}},
 'scan_leakage': {'leakage_threshold': 0.85}}

In [18]:
from etiq import Model


#log your dataset

dataset_loader = etiq.dataset(train)


#Log your already trained model

model = Model(model_architecture=standard_model, model_fitted=model_fit)


snapshot = project.snapshots.create(name="Test Snapshot", dataset=dataset_loader.initial_dataset, model=model, bias_params=dataset_loader.bias_params)


In [19]:
#scan_bias_sources

# Run our pipelines
(segments, issues, issue_summary) = snapshot.scan_bias_sources()

INFO:etiq.pipeline.DataPipeline0521:Starting pipeline
INFO:etiq.pipeline.DataPipeline0521:Computed metrics for the initial dataset
INFO:etiq.pipeline.DataPipeline0521:Completed pipeline
INFO:etiq.pipeline.DebiasPipeline0327:Starting pipeline
INFO:etiq.pipeline.DebiasPipeline0327:Start Phase IdentifyPipeline0622
INFO:etiq.pipeline.IdentifyPipeline0622:Using parent model
INFO:etiq.pipeline.IdentifyPipeline0622:Starting pipeline
INFO:etiq.pipeline.IdentifyPipeline0622:Checking proxy for feature age
INFO:etiq.pipeline.IdentifyPipeline0622:Checking correlation for feature age
INFO:etiq.pipeline.IdentifyPipeline0622:Checking proxy for feature educational-num
INFO:etiq.pipeline.IdentifyPipeline0622:Checking correlation for feature educational-num
INFO:etiq.pipeline.IdentifyPipeline0622:Checking proxy for feature fnlwgt
INFO:etiq.pipeline.IdentifyPipeline0622:Checking correlation for feature fnlwgt
INFO:etiq.pipeline.IdentifyPipeline0622:Checking proxy for feature capital-gain
INFO:etiq.pipeli

In [22]:
#if you want a shorter syntax for bias sources auto intead of using the config you can run the following:

etiq.load_config("./config_already_trained_bias_sources2.json")

{'dataset': {'label': 'income',
  'bias_params': {'protected': 'gender',
   'privileged': 1,
   'unprivileged': 0,
   'positive_outcome_label': 1,
   'negative_outcome_label': 0},
  'train_valid_test_splits': [1.0, 0.01, 0.0],
  'cat_col': 'cat_vars',
  'cont_col': 'cont_vars'},
 'scan_bias_sources': {'auto': True},
 'scan_accuracy_metrics': {'thresholds': {'accuracy': [0.8, 1.0],
   'true_pos_rate': [0.6, 1.0],
   'true_neg_rate': [0.6, 1.0]}},
 'scan_bias_metrics': {'thresholds': {'equal_opportunity': [0.0, 0.2],
   'demographic_parity': [0.0, 0.2],
   'equal_odds_tnr': [0.0, 0.2],
   'individual_fairness': [0.0, 0.8],
   'equal_odds_tpr': [0.0, 0.2]}},
 'scan_leakage': {'leakage_threshold': 0.85}}

In [23]:


#log your dataset

dataset_loader = etiq.dataset(train)
dl = etiq.dataset_loader.DatasetLoader(data=train, label='income', bias_params=dataset_loader.bias_params,
                   train_valid_test_splits=[1.0, 0.01, 0.0], cat_col=cat_vars,
                   cont_col=cont_vars, names_col = train.columns.values)

#Log your already trained model

model = Model(model_architecture=standard_model, model_fitted=model_fit)


snapshot = project.snapshots.create(name="Test Snapshot", dataset=dl.initial_dataset, model=model, bias_params=dataset_loader.bias_params)

snapshot.scan_bias_sources()

INFO:etiq.pipeline.DataPipeline0711:Starting pipeline
INFO:etiq.pipeline.DataPipeline0711:Computed metrics for the initial dataset
INFO:etiq.pipeline.DataPipeline0711:Completed pipeline
INFO:etiq.pipeline.DebiasPipeline0375:Starting pipeline
INFO:etiq.pipeline.DebiasPipeline0375:Start Phase IdentifyPipeline0974
INFO:etiq.pipeline.IdentifyPipeline0974:Using parent model
INFO:etiq.pipeline.IdentifyPipeline0974:Starting pipeline
INFO:etiq.pipeline.IdentifyPipeline0974:Checking proxy for feature age
INFO:etiq.pipeline.IdentifyPipeline0974:Checking correlation for feature age
INFO:etiq.pipeline.IdentifyPipeline0974:Checking proxy for feature educational-num
INFO:etiq.pipeline.IdentifyPipeline0974:Checking correlation for feature educational-num
INFO:etiq.pipeline.IdentifyPipeline0974:Checking proxy for feature fnlwgt
INFO:etiq.pipeline.IdentifyPipeline0974:Checking correlation for feature fnlwgt
INFO:etiq.pipeline.IdentifyPipeline0974:Checking proxy for feature capital-gain
INFO:etiq.pipeli

(    name                                      business_rule  \
 0      0  `occupation` == 3 and `native-country` == 39 a...   
 1      1            `occupation` == 14 and `workclass` == 4   
 2      2      `occupation` == 13 and `native-country` == 39   
 3      3                                  `occupation` == 6   
 4      4  `occupation` == 4 and `native-country` == 39 a...   
 5      5                                  `occupation` == 3   
 6      6  `native-country` == 39 and `occupation` == 3 a...   
 7      7  `native-country` == 39 and `occupation` == 3 a...   
 8      8      `native-country` == 39 and `occupation` == 14   
 9      9  `native-country` == 39 and `marital-status` ==...   
 10    10  `native-country` == 39 and `marital-status` ==...   
 11    11  `native-country` == 39 and `marital-status` ==...   
 12    12                                                all   
 13    13  `native-country` == 39 and `education` == 11 a...   
 14    14      `native-country` == 39 an